### Data Prep - July 28
#### Objective
1. Read in 4 csv files stored on box drive. For each...
1. Convert to dataframe
1. Remove rows missing XY daa
1. Convert remaining rows to spatial dataframe
1. Upload to AGOL

In [49]:
#Import packages
from datetime import datetime
import numpy as np
import pandas as pd
from arcgis import GIS
gis = GIS('pro')

In [2]:
#Check user
print (gis.users.me)

<User username:jpfay_dukeuniv>


In [36]:
#Read xlsx file listing filenames to process
df_files = pd.read_excel('Study and sample lists for ArcGIS Online.xlsx')

In [17]:
#Get folder location
box_folder = 'C:\\Users\\jpfay\\Box\\Files to Process\\USABLE Data\\NOT Processed\\'

In [57]:
#Initialize log file
str_now = datetime.now().strftime('%Y-%m-%d-%H-%M')
log_filename = f"log{str_now}.csv"
the_log = open(log_filename,'w')
the_log.write('Filename,Total Rows,Rows Processed\n')

'2020-07-29-09-03'

In [ ]:
idx = 0

In [64]:
row = df_files.loc[idx,:]
filename = box_folder+row['File name']
lat_fld = row['Latitude Field Name']
lng_fld = row['Longitude Field Name']
hg_fld = row['Hg Field Name']
sample_type = row['Sample type']

#Read in file
df = pd.read_csv(filename,
                 usecols=[lat_fld,lng_fld,hg_fld],
                 dtype='float')
total_rows = df.shape[0]
df_files.loc[idx,'total records'] = total_rows
print(f'{total_rows} total rows')

#Remove rows missing values
df_valid = df.dropna(axis='rows',how='any')
valid_rows = df_valid.shape[0]
df_files.loc[idx,'valid records'] = valid_rows
print(f'{valid_rows} valid rows')

3259 total rows
3259 valid rows


In [66]:
#Convert to spatial dataframe
sdf = pd.DataFrame.spatial.from_xy(
    df = df_valid,
    x_column=lng_fld,
    y_column=lat_fld)

In [68]:
sdf.spatial.plot()

MapView(layout=Layout(height='400px', width='100%'))

In [ ]:
#Convert to feature layer